# This is a compilation of functions used within this project

In [ ]:
def latlonbin(invar,lat,lon,date,binwidth):
    import numpy as np
    
    # create a pandas dataframe
    df = pd.DataFrame(dict(
            invar = np.array(invar),
            lat= np.array(lat),
            lon= np.array(lon),
            date = np.array(date)
        ))
    
    # # add in a column for the month
    df['month'] = pd.DatetimeIndex(df['date']).month

    # create 1 degree bins
    latedges = np.arange(lat.min()-(binwidth/2),lat.max()+(binwidth/2),binwidth)
    lat_inds = list(range(len(latedges)-1))

    lonedges = np.arange(lon.min()-(binwidth/2),lon.max()+(binwidth/2),binwidth)
    lon_inds = list(range(len(lonedges)-1))

    latbins = latedges[1:]-(binwidth/2)
    lonbins = lonedges[1:]-(binwidth/2)

    df['latedges'] = pd.cut(lat, latedges)
    df['lonedges'] = pd.cut(lon, lonedges)
    df['latbins_ind'] = pd.cut(lat, latedges,labels = lat_inds)
    df['lonbins_ind'] = pd.cut(lon, lonedges,labels = lon_inds)
    df['lat_lon_indx']=df.groupby(['latbins_ind', 'lonbins_ind']).ngroup()
    grouped = df.groupby(['latbins_ind', 'lonbins_ind'])

    invar_BINNED = np.zeros((len(latbins),len(lonbins)), dtype=np.ndarray)
    invar_BINNED[:] = np.nan

    invar_binned_ave = np.zeros((len(latbins),len(lonbins)), dtype=np.ndarray)
    invar_binned_ave[:] = np.nan


    #extract the data for each group
    for name, group in grouped:
        i = np.array(group.latbins_ind)
        j = np.array(group.lonbins_ind)

        invar_BINNED[i[0],j[0]] = group.invar

        invar_binned_ave[i[0],j[0]] = np.nanmean(group.invar)
        
    # create pcolormesh lat/lon vals
    
    # extend longitude by 2
    lon_extend = np.zeros(lonbins.size+2)
    # fill in internal values
    lon_extend[1:-1] = lonbins # fill up with original values
    # fill in extra endpoints
    lon_extend[0] = lonbins[0]-np.diff(lonbins)[0]
    lon_extend[-1] = lonbins[-1]+np.diff(lonbins)[-1]
    # calculate the midpoints
    lon_pcolormesh_midpoints = lon_extend[:-1]+0.5*(np.diff(lon_extend))

    # extend latitude by 2
    lat_extend = np.zeros(latbins.size+2)
    # fill in internal values
    lat_extend[1:-1] = latbins
    # fill in extra endpoints
    lat_extend[0] = latbins[0]-np.diff(latbins)[0]
    lat_extend[-1] = latbins[-1]+np.diff(latbins)[-1]
    # calculate the midpoints
    lat_pcolormesh_midpoints = lat_extend[:-1]+0.5*(np.diff(lat_extend))
    

    return(np.array(invar_binned_ave,dtype = float), 
           lonbins,latbins,lon_pcolormesh_midpoints,lat_pcolormesh_midpoints)

In [ ]:
def add_land(ax,bounds):
    from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
    ax.add_feature(cfeature.LAND,color='gray',zorder = 2)
    ax.background_patch.set_facecolor('k')
    ax.coastlines(resolution='110m',zorder = 2)
    g = ax.gridlines(draw_labels=True,alpha=0)
    g.xlabels_top = False
    g.ylabels_right = False
    g.xlabel_style = {'size': 15}
    g.ylabel_style = {'size': 15}
    g.xformatter = LONGITUDE_FORMATTER
    g.yformatter = LATITUDE_FORMATTER
    ax.axes.axis('tight')
    ax.set_extent(bounds, crs=ccrs.PlateCarree())
    
